In [7]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats
import seaborn.apionly as sns

SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral":"Adipose.V",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "SkinUnexposed",
    "Skin-SunExposed":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}
for T in sorted(SHORTEN.keys()):
    summary = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+T+'/Lin_Reg_Out', sep='\t')
    print(list(summary.columns[:6]), T,summary.shape ,'\n')
#summary

In [4]:
summary.transpose()

,0,1,2,3,4,5,6,7,8,9,...,81929,81930,81931,81932,81933,81934,81935,81936,81937,81938
gene,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,...,ENSG00000242616.3,ENSG00000242616.3,ENSG00000242616.3,ENSG00000244187.3,ENSG00000260230.2,ENSG00000260230.2,ENSG00000260230.2,ENSG00000268996.1,ENSG00000268996.1,ENSG00000268996.1
chrom,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,...,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9
str.id,STR_169735051,STR_169784101,STR_169895824,STR_169937973,STR_169614029,STR_169667612,STR_169701424,STR_169701512,STR_169735051,STR_169784101,...,STR_114404884,STR_114441796,STR_114511252,STR_139666301,STR_111891767,STR_111937749,STR_111983014,STR_139894040,STR_140014401,STR_140029812
str.start,169735051,169784101,169895824,169937973,169614029,169667612,169701424,169701512,169735051,169784101,...,114404884,114441796,114511252,139666301,111891767,111937749,111983014,139894040,140014401,140029812
n.miss,51,45,50,38,18,68,14,26,51,45,...,56,49,57,65,12,64,8,63,48,48
allele1.dummy,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
allele2.dummy,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
af.dummy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beta,0.044187,0.0179004,0.0131296,-0.019647,0.0517572,0.134485,-0.0636871,-0.0485744,0.0482733,-0.0205681,...,-0.0666603,-0.105726,-0.00742832,-0.0947223,0.0879198,-0.0326247,0.0592442,-0.00839826,0.0617559,-0.0325922
beta.se,0.0593858,0.0588141,0.059334,0.0581126,0.056179,0.0607569,0.0557882,0.056913,0.0593745,0.0588111,...,0.0598426,0.0589029,0.0600825,0.0606969,0.055512,0.0608257,0.0552876,0.0607435,0.0590184,0.0590998


In [4]:
linreg = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/WholeBlood/Lin_Reg_Out', sep='\t')
linreg.transpose()

,0,1,2,3,4,5,6,7,8,9,...,81929,81930,81931,81932,81933,81934,81935,81936,81937,81938
gene,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,...,ENSG00000242616.3,ENSG00000242616.3,ENSG00000242616.3,ENSG00000244187.3,ENSG00000260230.2,ENSG00000260230.2,ENSG00000260230.2,ENSG00000268996.1,ENSG00000268996.1,ENSG00000268996.1
chrom,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,...,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9
str.id,STR_169735051,STR_169784101,STR_169895824,STR_169937973,STR_169614029,STR_169667612,STR_169701424,STR_169701512,STR_169735051,STR_169784101,...,STR_114404884,STR_114441796,STR_114511252,STR_139666301,STR_111891767,STR_111937749,STR_111983014,STR_139894040,STR_140014401,STR_140029812
str.start,169735051,169784101,169895824,169937973,169614029,169667612,169701424,169701512,169735051,169784101,...,114404884,114441796,114511252,139666301,111891767,111937749,111983014,139894040,140014401,140029812
n.miss,51,45,50,38,18,68,14,26,51,45,...,56,49,57,65,12,64,8,63,48,48
allele1.dummy,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
allele2.dummy,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
af.dummy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beta,0.044187,0.0179004,0.0131296,-0.019647,0.0517572,0.134485,-0.0636871,-0.0485744,0.0482733,-0.0205681,...,-0.0666603,-0.105726,-0.00742832,-0.0947223,0.0879198,-0.0326247,0.0592442,-0.00839826,0.0617559,-0.0325922
beta.se,0.0593858,0.0588141,0.059334,0.0581126,0.056179,0.0607569,0.0557882,0.056913,0.0593745,0.0588111,...,0.0598426,0.0589029,0.0600825,0.0606969,0.055512,0.0608257,0.0552876,0.0607435,0.0590184,0.0590998


In [ ]:

#TODO
join ${OUTDIR}/${TISSUE}_${PREFIX}_enrich_causal.tab  ${OUTDIR}/${TISSUE}_${PREFIX}_enrich.tab, 
        output category and odds_ratio_causal/odds_ratio_allestr - TODO
    
join -t $'\t' ${TMPDIR}/${TISSUE}_${PREFIX}_eSTR_overlap.tab ${TMPDIR}/${TISSUE}_${PREFIX}_allSTR_overlap.tab    

    
    
    #82915